In [1]:
import json

import pandas as pd

PLANTS_PARAM = json.load(open("../../resources/solar_plants.json"))

SOLAR_PLANT = "Hélio"

In [2]:
equation = PLANTS_PARAM[SOLAR_PLANT]["equation"]

print("Perda estimada = " + equation)

loss_table = pd.read_parquet(
    "../../" + PLANTS_PARAM[SOLAR_PLANT]["datawarehouse"]["loss_table"]
)

independent_vars = loss_table[["CSI", "Angulação (°)"]]
dependent_var = loss_table["Perda (%)"]

Perda estimada = 3.74+(16.77*CSI)+(0.22*Ângulo)+(1.68*CSI^2)-(0.19*CSI*Ângulo)+(0.01*Ângulo^2)


In [3]:
estimated_loss = pd.DataFrame(columns=["Perda estimada (%)"])

for csi, angle in independent_vars.values:
    aux_equation = (
        equation.replace("CSI", str(csi))
        .replace("Ângulo", str(angle))
        .replace("^", "**")
    )

    loss = pd.DataFrame({"Perda estimada (%)": eval(aux_equation)}, index=[0])

    if estimated_loss.empty:
        estimated_loss = loss
    else:
        estimated_loss = pd.concat([estimated_loss, loss], ignore_index=True)

estimated_loss

,Perda estimada (%)
0,-21.230968
1,-15.159968
2,-9.588968
3,-4.517968
4,0.053032
...,...
8720,35.667148
8721,40.361648
8722,45.556148
8723,51.250648


In [4]:
compared_loss = pd.concat([dependent_var, estimated_loss], axis=1)

diff = round(abs(compared_loss["Perda (%)"] - compared_loss["Perda estimada (%)"]), 2)

mean_precision = round(diff.mean(), 2)

print(f"Precisão média: {100 - mean_precision}%")

Precisão média: 85.99%
